In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import metrics

go_back = '../data/'

2024-04-12 19:39:02.621524: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv(go_back + 'all_data_train.csv')
df.head()

,Unnamed: 0,date,team,opponent,score,opponentScore,moneyLine,opponentMoneyLine,total,spread,...,REB/G_opp,AST/G_opp,STL/G_opp,BLK/G_opp,TOV/G_opp,PF/G_opp,PTS/G_opp,win%_opp,home,outcome
0,0,2008-10-31,Chicago,Boston,80,96,450.0,-600.0,189.0,10.0,...,36.0,16.0,10.0,0.0,16.0,28.0,90.0,1.0,0,0
1,1,2008-10-31,Boston,Chicago,96,80,-600.0,450.0,189.0,-10.0,...,40.0,27.0,9.0,6.0,18.0,18.0,108.0,1.0,1,1
2,2,2008-10-31,LA Clippers,Denver,103,113,100.0,-120.0,196.5,1.5,...,35.0,17.0,12.0,3.0,13.0,27.0,94.0,0.0,1,0
3,3,2008-10-31,Toronto,Golden State,112,108,-280.0,240.0,207.0,-6.5,...,39.0,18.0,10.0,5.0,13.0,26.0,103.0,0.0,1,1
4,4,2008-10-31,Denver,LA Clippers,113,103,-120.0,100.0,196.5,-1.5,...,32.0,24.0,9.0,5.0,18.0,29.0,79.0,0.0,0,1


In [3]:
df['actual_spread'] = df['opponentScore'] - df['score']

In [4]:
X = df.drop(['Unnamed: 0', 'team', 'opponent', 'total', 'date', 'score',
            'opponentScore','secondHalfTotal', 'total', 'actual_spread', 'outcome'], axis = 1)
Y = df['actual_spread']

X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

In [5]:
test = pd.read_csv(go_back + 'all_data_test.csv')
test['actual_spread'] = test['opponentScore'] - test['score']

X_test = test.drop(['Unnamed: 0', 'team', 'opponent', 'total', 'date', 'score',
            'opponentScore','secondHalfTotal', 'total', 'actual_spread', 'outcome'], axis = 1)
y_test = test['actual_spread']

# Regression Modeling

In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
model = LinearRegression(fit_intercept = True)
model.fit(X_train, y_train)

lr_train_pred = model.predict(X_train)
train_rmse = np.sqrt(mean_squared_error(y_train, lr_train_pred))

lr_val_pred = model.predict(X_val)
val_mse = mean_squared_error(y_val, lr_val_pred)
val_rmse = np.sqrt(val_mse)

lr_test_pred = model.predict(X_test)
test_rmse = np.sqrt(mean_squared_error(y_test, lr_test_pred))

print('Train RMSE:', train_rmse)
print("Val RMSE:", val_rmse)
print('Test RMSE:', test_rmse)

Train RMSE: 12.047873509843477
Val RMSE: 12.122899232571793
Test RMSE: 13.603670330785382


In [7]:
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(max_depth = 6, min_samples_leaf = 3)
rf_model.fit(X_train, y_train)

rf_train_pred = rf_model.predict(X_train)
train_rmse = np.sqrt(mean_squared_error(y_train, rf_train_pred))

rf_val_pred = rf_model.predict(X_val)
val_mse = mean_squared_error(y_val, rf_val_pred)
val_rmse = np.sqrt(val_mse)

rf_test_pred = rf_model.predict(X_test)
test_rmse = np.sqrt(mean_squared_error(y_test, rf_test_pred))

print('Train RMSE:', train_rmse)
print("Val RMSE:", val_rmse)
print('Test RMSE:', test_rmse)

Train RMSE: 11.74517686774626
Val RMSE: 12.108525009975082
Test RMSE: 13.651822544055161


In [8]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
from sklearn.neural_network import MLPRegressor

MLP_model = MLPRegressor(early_stopping=True, solver="adam", max_iter=1000)

MLP_model.fit(X_train, y_train)

mlp_train_pred = MLP_model.predict(X_train)
train_rmse = np.sqrt(mean_squared_error(y_train, mlp_train_pred))

mlp_val_pred = MLP_model.predict(X_val)
val_mse = mean_squared_error(y_val, mlp_val_pred)
val_rmse = np.sqrt(val_mse)

mlp_test_pred = MLP_model.predict(X_test)
test_rmse = np.sqrt(mean_squared_error(y_test, mlp_test_pred))

print('Train RMSE:', train_rmse)
print("Val RMSE:", val_rmse)
print('Test RMSE:', test_rmse)

Train RMSE: 12.1420237625337
Val RMSE: 12.271960469090311
Test RMSE: 13.723654129522323


# Calculating Expected Earnings

In [10]:
vegas_spread = list(test['spread'])

In [11]:
def spread_earnings(preds, actual, vegas):
    earnings = 0
    num_correct = 0
    for i in range(len(preds)):
        if (preds[i] > vegas[i]) and (actual[i] > vegas[i]):
            earnings += 100
            num_correct += 1
        elif (preds[i] < vegas[i]) and (actual[i] < vegas[i]):
            earnings += 100
            num_correct += 1
        elif actual[i] == vegas[i]:
            continue
        else:
            earnings -= 110
    return earnings, num_correct / len(preds)

In [12]:
temp = test[['opponentScore', 'score', 'spread', 'actual_spread']]
temp['preds'] = lr_test_pred
temp

,opponentScore,score,spread,actual_spread,preds
0,83,115,7.0,-32,5.482364
1,128,112,7.0,16,6.968905
2,102,96,7.5,6,7.659390
3,124,91,2.5,33,2.606027
4,134,135,2.0,-1,0.956377
...,...,...,...,...,...
1951,118,106,6.5,12,5.553931
1952,80,111,-17.0,-31,-16.604898
1953,120,130,-10.0,-10,-9.430541
1954,94,105,0.0,-11,0.006118


In [13]:
spread_earnings(list(lr_test_pred), list(y_test), vegas_spread)

(-1420, 0.5112474437627812)

In [14]:
spread_earnings(list(rf_test_pred), list(y_test), vegas_spread)

(-9610, 0.49130879345603273)

In [15]:
spread_earnings(list(mlp_test_pred), list(y_test), vegas_spread)

(-12760, 0.483640081799591)